In [ ]:
pip install dotenv langchain langchain_core

In [ ]:
pip install -U langchain-google-genai

In [8]:
import getpass
import os

from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

In [12]:
load_dotenv()

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai", temperature=0)
model

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x10b3b71a0>, default_metadata=())

In [13]:
messages = [
    SystemMessage("Translate the following from English into Hindko"),
    HumanMessage("Hi!"),
]

print(model.invoke(messages).content)

There isn't a single, universally accepted translation of "Hi!" in Hindko, as it depends on the context and the relationship between the speakers. Here are a few options:

*   **Assalam-o-Alaikum (السلام علیکم):** This is the most common and respectful greeting, used in most situations. It means "Peace be upon you."

*   **Adaab (آداب):** This is another respectful greeting, similar to "Hello" or "Greetings."

*   **Kia haal hai? (کیا حال ہے؟):** This translates to "How are you?" and can be used as a greeting.

*   **Theek ho? (ٹھیک ہو؟):** This translates to "Are you well?" and can also be used as a greeting, especially among friends.

*   **Koi nawaan? (کوئی نواں؟):** This translates to "What's new?" and can be used as a casual greeting.

So, the best translation depends on the situation. If you want to be polite and respectful, use **Assalam-o-Alaikum** or **Adaab**. If you're talking to a friend, you could use **Kia haal hai?**, **Theek ho?**, or **Koi nawaan?**.


# Streaming
Streaming in LangChain refers to the ability to process and return data incrementally as it becomes available, rather than waiting for the entire response to be generated before returning it.

1. Large Language Model (LLM) Responses:* When generating long text outputs, streaming allows you to show tokens as they're generated

2. Real-time Processing: For applications that need to display or process information as it arrives

3. Improved User Experience: Users see progress instead of waiting for complete responses

## How Streaming Works in LangChain

LangChain provides streaming capabilities through:

1. Streaming Callbacks: Implementing callback handlers that process chunks of data as they arrive

2. Streaming Interfaces: Special interfaces for models that support streaming outputs

## Key Benefits
Reduced Latency: Users see the first parts of the response immediately

Memory Efficiency: Doesn't require storing the entire response in memory at once

Interactive Experience: Enables more conversational interfaces

Streaming is especially valuable for chat applications, real-time data processing, and any scenario where immediate feedback is important.

In [14]:
for token in model.stream("Explain the concept of streaming in AI systems"):
    print(token.content, end="|")

In| the context of AI systems, **streaming** refers to the continuous processing of data as it arrives|, rather than waiting for a complete dataset to be collected before processing begins. Think| of it like a river flowing continuously, instead of a lake that needs to fill up before you can use the water.

Here's a breakdown of the concept:|

**Key Characteristics of Streaming in AI:**

 processing where data is collected and processed in discrete chunks. stream.  This is in contrast to batch|
*   **Real-time or Near Real-time Processing:**  The goal is to process data as quickly as possible after it arrives, enabling timely insights and actions.
 Latency:**  Minimizing the delay between data arrival and processing completion is crucial.
*   **Scalability:**  The system must be able to handle varying data volumes and processing demands without significant performance degradation.
 Management:**  Streaming systems often need to maintain some form of state (e.g., running averages, count

# Prompt Template from Messages in LangChain

## Overview
The `PromptTemplate.from_messages()` method creates structured conversation prompts by combining multiple message components with different roles. This is essential for chat models requiring conversation history or role-specific prompts.

Best Practices
Always include system message for initial instructions

Use variables to make templates reusable

Maintain proper message ordering (system → history → human)

For long conversations, consider using MessagesPlaceholder

In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a {subject} tutor."),
    ("human", "Explain {topic} in simple terms")
])

formatted = chat_prompt.format_messages(
    subject="math",
    topic="quadratic equations"
    
)
print("Prompt template using format message,",formatted[0].content)
print("Prompt template using format message,",formatted[1].content)


# OR

system_template = "Translate the following from English into {language}"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

prompt = prompt_template.invoke({"language": "arabic", "text": "hi!"})
response = model.invoke(prompt)
print(response.content)




Prompt template using format message, You are a math tutor.
Prompt template using format message, Explain quadratic equations in simple terms
أهلاً! (Ahlan!)

This is a general and friendly greeting. You could also say:

*   مرحباً (Marhaban) - Also a common greeting.
*   السلام عليكم (As-salamu alaykum) - A more formal and traditional greeting, meaning "Peace be upon you."


# PromptTemplate.from_strings() in LangChain

## Overview
The `PromptTemplate.from_strings()` method creates a prompt template from a sequence of string templates. This is useful for constructing prompts from multiple text components that need to be combined.

## Key Features
- Combines multiple string fragments into a single prompt

- Supports template variables in individual strings

- Preserves the exact ordering of input strings

- Automatically joins strings with spaces by default

In [44]:
prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")
prompt = prompt_template.invoke({"topic": "cats"})
response = model.invoke(prompt)
print(response.content)

Why did the cat join the Red Cross?

Because he wanted to be a first-aid kit!


In [45]:
prompt_template = PromptTemplate.from_template("Give me one good facts about {wrestler_name} in {wrestling_company}")
prompt =  prompt_template.invoke({"wrestler_name":"roman reigns" ,"wrestling_company":"wwe"})
response = model.invoke(prompt)
print(response.content)

One impressive fact about Roman Reigns in WWE is that he holds the record for the longest Universal Championship reign at 1,316 days.


# `from_template()` vs `from_messages()` in LangChain

## Key Differences

| Feature                | `from_template()`                            | `from_messages()`                            |
|------------------------|---------------------------------------------|---------------------------------------------|
| **Primary Use Case**   | Simple single-text prompts                  | Structured conversation prompts             |
| **Input Format**       | Single string template                      | List of (role, content) tuples              |
| **Role Support**       | No role separation                          | Explicit roles (system/human/ai/custom)     |
| **Variable Handling**  | Variables in one template                   | Variables can be role-specific              |
| **Output Structure**   | Flat text prompt                            | Structured chat history format              |
| **Best For**           | One-off queries, simple LLM interactions    | Chat applications, multi-turn conversations |



# Runnable Interfaces in LangChain

## Overview
Runnables are the fundamental building blocks in LangChain that encapsulate executable units of work. They provide a standardized interface for operations that process input data to produce output.

## Core Concepts

### Key Characteristics
- **Uniform Interface**: All Runnables implement `.invoke()`, `.stream()`, and `.batch()`
- **Composable**: Can be chained together using the pipe (`|`) operator
- **Async Support**: Native asynchronous execution
- **Batch Processing**: Efficient handling of multiple inputs

In [59]:
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model | StrOutputParser()
print(chain.invoke({"topic": "uncles"}))

Why did the scarecrow win an award?

Because he was outstanding in his field! My Uncle told me that one. He's always corny.


## Coercion
We can even combine this chain with more runnables to create another chain. This may involve some input/output formatting using other types of runnables, depending on the required inputs and outputs of the chain components.

For example, let's say we wanted to compose the joke generating chain with another chain that evaluates whether or not the generated joke was funny.

In [60]:
analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")
composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()
print(composed_chain.invoke({"topic": "uncles"}))

Yes, that's a classic pun and a funny joke! Here's why:

*   **Pun:** The humor comes from the double meaning of "outstanding in his field." It literally means he's physically standing out in a field, but it also means he's exceptionally good at what he does.
*   **Corny:** It's definitely a corny joke, which adds to the humor. The fact that your uncle is "always corny" makes it even funnier because it fits his personality.
*   **Relatability:** Most people have heard similar puns, so it's easily understood and appreciated.

So, yes, it's a funny joke, especially because of its corny nature and the context of your uncle telling it.


# RunnableParallel in LangChain

## Overview
`RunnableParallel` (formerly known as `RunnableMap`) enables parallel execution of multiple Runnables. It takes a single input and distributes it to several Runnables simultaneously, returning a dictionary of their outputs.

## Key Features
- **Parallel Execution**: Runs multiple Runnables concurrently
- **Dictionary Output**: Returns results in a structured format
- **Input Broadcasting**: Same input sent to all branches
- **Type Safety**: Maintains input/output type signatures


In [76]:
from langchain_core.runnables import RunnableLambda

def add_five(x): return x + 5
sequence = RunnableLambda(add_five) | {"double": lambda x: x*2, "triple": lambda x: x*3}
print(sequence.invoke(10))  # What happens?

{'double': 30, 'triple': 45}


In [77]:


def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
sequence = runnable_1 | runnable_2
# Or equivalently:
# sequence = RunnableSequence(first=runnable_1, last=runnable_2)
sequence.invoke(1)
await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

[4, 6, 8]

In [73]:

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

def mul_three(x: int) -> int:
    return x * 3

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)

sequence = runnable_1 | {  # this dict is coerced to a RunnableParallel
    "mul_two": runnable_2,
    "mul_three": runnable_3,
}

# Or equivalently:
# sequence = runnable_1 | RunnableParallel(
#     {"mul_two": runnable_2, "mul_three": runnable_3}
# )
# Also equivalently:
# sequence = runnable_1 | RunnableParallel(
#     mul_two=runnable_2,
#     mul_three=runnable_3,
# )

sequence.invoke(1)
await sequence.ainvoke(1)

sequence.batch([1, 2, 3])
await sequence.abatch([1, 2, 3])

[{'mul_two': 4, 'mul_three': 6},
 {'mul_two': 6, 'mul_three': 9},
 {'mul_two': 8, 'mul_three': 12}]

# RunnableBranch in LangChain

## Overview
`RunnableBranch` provides conditional routing within LangChain workflows, allowing different Runnables to be executed based on input conditions.

## Basic Syntax
```python
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
    (condition1, runnable1),
    (condition2, runnable2),
    default_runnable
)

In [81]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: isinstance(x, int), lambda x: x + 1),
    (lambda x: isinstance(x, float), lambda x: x * 2),
    lambda x: "goodbye",
)

branch.invoke(3) 

4

In [82]:
branch.invoke("hello") # "HELLO"

'HELLO'

In [83]:
branch.invoke(None) # "goodbye"

'goodbye'